In [ ]:
import yfinance as yf
import plotly.graph_objs as go
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.dropna(inplace=True)
    return stock_data

def preprocess_data(stock_data, look_back):
    data = stock_data.filter(['Close']).values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    x_train, y_train = [], []
    for i in range(look_back, len(scaled_data)):
        x_train.append(scaled_data[i - look_back:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))

    return x_train, y_train, scaler

def create_lstm_model(input_shape):
    model = Sequential()
    model.add(LSTM(50, return_sequences=True, input_shape=input_shape))
    model.add(LSTM(50, return_sequences=False))
    model.add(Dense(25))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def predict_stock_price(model, stock_data, look_back, scaler):
    last_days = stock_data.filter(['Close']).values[-look_back:]
    last_days_scaled = scaler.transform(last_days)

    X_test = np.array([last_days_scaled])
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    predicted_price = model.predict(X_test)
    predicted_price = scaler.inverse_transform(predicted_price)

    return predicted_price[0][0]

def plot_candlestick_chart(stock_data, ticker, predicted_price):
    fig = go.Figure()

    # Add candlestick chart
    fig.add_trace(go.Candlestick(x=stock_data.index,
                                 open=stock_data['Open'],
                                 high=stock_data['High'],
                                 low=stock_data['Low'],
                                 close=stock_data['Close'],
                                 name='Stock Price'))

    # Add predicted closing price
    fig.add_trace(go.Scatter(x=[stock_data.index[-1], stock_data.index[-1] + pd.DateOffset(days=1)],
                             y=[stock_data['Close'][-1], predicted_price],
                             mode='lines+markers',
                             name='Predicted Closing Price',marker_color='rgba(255, 182, 193, .9)'))

    fig.update_layout(title=f'{ticker} Stock Price',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)
    fig.show()

if __name__ == "__main__":
    ticker = 'AAPL'
    start_date = '2022-05-01'
    end_date = datetime.today().strftime('%Y-%m-%d')

    look_back = 60

    stock_data = fetch_stock_data(ticker, start_date, end_date)
    x_train, y_train, scaler = preprocess_data(stock_data, look_back)

    model = create_lstm_model((x_train.shape[1], 1))
    model.fit(x_train, y_train, batch_size=1, epochs=5)

    predicted_price = predict_stock_price(model, stock_data,look_back, scaler)
    print(f"Predicted closing price for the next trading day: {predicted_price}")

    plot_candlestick_chart(stock_data, ticker, predicted_price)


[*********************100%***********************]  1 of 1 completed
Epoch 1/5
215/215 [==============================] - 12s 37ms/step - loss: 0.0208
Epoch 2/5
215/215 [==============================] - 13s 59ms/step - loss: 0.0083
Epoch 3/5
215/215 [==============================] - 12s 57ms/step - loss: 0.0087
Epoch 4/5
215/215 [==============================] - 6s 27ms/step - loss: 0.0059
Epoch 5/5
1/1 [==============================] - 1s 812ms/step
Predicted closing price for the next trading day: 177.71372985839844


In [ ]:
import yfinance as yf
import plotly.graph_objs as go
from datetime import datetime
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

def fetch_stock_data(ticker, start_date, end_date):
    stock_data = yf.download(ticker, start=start_date, end=end_date)
    stock_data.dropna(inplace=True)
    return stock_data

def preprocess_data(stock_data, look_back):
    data = stock_data.filter(['Close']).values
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data)

    x_train, y_train = [], []
    for i in range(look_back, len(scaled_data)):
        x_train.append(scaled_data[i - look_back:i, 0])
        y_train.append(scaled_data[i, 0])

    x_train, y_train = np.array(x_train), np.array(y_train)

    return x_train, y_train, scaler

def create_ann_model(input_shape):
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=input_shape))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(1))

    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def predict_stock_price(model, stock_data, look_back, scaler):
    last_days = stock_data.filter(['Close']).values[-look_back:]
    last_days_scaled = scaler.transform(last_days)

    X_test = np.array([last_days_scaled])
    predicted_price = model.predict(X_test)
    predicted_price = scaler.inverse_transform(predicted_price)

    return predicted_price[0][0]

def plot_candlestick_chart(stock_data, ticker, predicted_price):
    fig = go.Figure()

    # Add candlestick chart
    fig.add_trace(go.Candlestick(x=stock_data.index,
                                 open=stock_data['Open'],
                                 high=stock_data['High'],
                                 low=stock_data['Low'],
                                 close=stock_data['Close'],
                                 name='Stock Price'))

    # Add predicted closing price
    fig.add_trace(go.Scatter(x=[stock_data.index[-1], stock_data.index[-1] + pd.DateOffset(days=1)],
                             y=[stock_data['Close'][-1], predicted_price],
                             mode='lines+markers',
                             name='Predicted Closing Price'))

    fig.update_layout(title=f'{ticker} Stock Price',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      xaxis_rangeslider_visible=False)
    fig.show()

if __name__ == "__main__":
    ticker = 'AAPL'
    start_date = '2020-01-01'
    end_date = datetime.today().strftime('%Y-%m-%d')

    look_back = 60

    stock_data = fetch_stock_data(ticker, start_date, end_date)
    x_train, y_train, scaler = preprocess_data(stock_data, look_back)

    model = create_ann_model((x_train.shape[1],))
    model.fit(x_train, y_train, batch_size=1, epochs=5)

    predicted_price = predict_stock_price(model, stock_data, look_back, scaler)
    print(f"Predicted closing price for the next trading day: {predicted_price}")

    plot_candlestick_chart(stock_data, ticker, predicted_price)


[*********************100%***********************]  1 of 1 completed
Epoch 1/5
802/802 [==============================] - 2s 1ms/step - loss: 0.0054
Epoch 2/5
802/802 [==============================] - 1s 2ms/step - loss: 0.0032
Epoch 3/5
802/802 [==============================] - 1s 2ms/step - loss: 0.0027
Epoch 4/5
802/802 [==============================] - 1s 1ms/step - loss: 0.0024
Epoch 5/5
1/1 [==============================] - 0s 86ms/step
Predicted closing price for the next trading day: 174.66017150878906


In [ ]:
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
today = date.today()

d1 = today.strftime("%Y-%m-%d")
end_date = d1
d2 = date.today() - timedelta(days=365)
d2 = d2.strftime("%Y-%m-%d")
start_date = d2

data = yf.download('SBIN.NS',
                   start=start_date,
                   end=end_date,
                   progress=False)

data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-06-06,464.200012,465.600006,456.500000,463.700012,454.860901,10661432
2022-06-07,461.850006,465.950012,460.500000,463.399994,454.566620,7672515
2022-06-08,466.600006,475.549988,463.200012,471.299988,462.316010,17168871
2022-06-09,468.950012,470.899994,462.450012,466.950012,458.048950,12635601
2022-06-10,462.000000,464.149994,458.600006,461.850006,453.046173,8621104
...,...,...,...,...,...,...
2023-05-30,595.000000,595.500000,591.500000,592.799988,581.500000,15031640
2023-05-31,583.799988,584.799988,577.500000,579.849976,579.849976,28797643
2023-06-01,581.799988,586.400024,581.000000,582.750000,582.750000,17820226


In [ ]:
data["Date"] = data.index
data = data[["Date", "Open", "High", "Low", "Close", "Adj Close", "Volume"]]
data.reset_index(drop=True, inplace=True)

model = LinearRegression()

X_train = data.index.values.reshape(-1, 1)
y_train = data['Close'].values.reshape(-1, 1)
model.fit(X_train, y_train)

num_days = 30
future_dates = pd.date_range(start=d1, periods=num_days, freq='D')
X_test = [(date - data['Date'][0]).days for date in future_dates]
X_test = pd.DataFrame(X_test).values.reshape(-1, 1)
y_pred = model.predict(X_test)

fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                                      open=data['Open'], high=data['High'],
                                      low=data['Low'], close=data['Close'])])
fig.add_trace(go.Scatter(x=future_dates, y=y_pred.flatten(), mode='lines', name='Future Prices'))
fig.update_layout(title='Share Price Analysis with Future Price Prediction', xaxis_rangeslider_visible=False)
fig.show()

fig = px.line(data, x=data.Date, y=data.Close, title='Daily Stock Prices: Line Chart')
fig.show()

In [ ]:
window_size = 20
sma = data['Close'].rolling(window=window_size).mean()

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=data.Date,
        y=sma
    ))

fig.add_trace(
    go.Bar(
        x=data.Date,
        y=data.Close
    ))

fig.show()

In [ ]:
window_size = 20
ema= data['Close'].ewm(span=window_size, adjust=False).mean()

fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=data.Date,
        y=ema
    ))

fig.add_trace(
    go.Bar(
        x=data.Date,
        y=data.Close
    ))

fig.show()

In [ ]:
#Defining the support and resistance levels
support_levels = data[(data['Low'].shift(1) > data['Low']) & (data['Low'].shift(-1) > data['Low'])]
resistance_levels = data[(data['High'].shift(1) < data['High']) & (data['High'].shift(-1) < data['High'])]
print("Support Levels:")
print(support_levels[['Date', 'Low']])
print("\nResistance Levels:")
print(resistance_levels[['Date', 'High']])

Support Levels:
          Date         Low
5   2022-06-13  442.100006
10  2022-06-20  430.700012
26  2022-07-12  483.100006
29  2022-07-15  474.700012
35  2022-07-25  509.149994
37  2022-07-27  512.250000
40  2022-08-01  526.400024
43  2022-08-04  524.200012
46  2022-08-10  513.400024
50  2022-08-17  524.349976
54  2022-08-23  508.250000
58  2022-08-29  508.750000
64  2022-09-07  531.549988
69  2022-09-14  549.250000
72  2022-09-19  559.549988
77  2022-09-26  533.000000
81  2022-09-30  516.150024
86  2022-10-10  522.200012
89  2022-10-13  520.200012
91  2022-10-17  524.200012
94  2022-10-20  545.299988
99  2022-10-28  567.000000
103 2022-11-03  568.000000
109 2022-11-14  591.650024
114 2022-11-21  595.349976
119 2022-11-28  602.349976
121 2022-11-30  598.400024
126 2022-12-07  604.500000
133 2022-12-16  602.099976
139 2022-12-26  570.700012
142 2022-12-29  597.150024
148 2023-01-06  597.000000
150 2023-01-10  590.599976
152 2023-01-12  590.099976
155 2023-01-17  586.049988
157 2023-01-

In [ ]:
#Marking Support and Resistance Levels and Chart
support_levels = data[(data['Low'].shift(1) > data['Low']) & (data['Low'].shift(-1) > data['Low'])]

resistance_levels = data[(data['High'].shift(1) < data['High']) & (data['High'].shift(-1) < data['High'])]

fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'])])

fig.add_trace(go.Scatter(x=support_levels['Date'],
                         y=support_levels['Low'],
                         mode='markers',
                         name='Support Levels',
                         marker=dict(color='blue', size=5)))

fig.add_trace(go.Scatter(x=resistance_levels['Date'],
                         y=resistance_levels['High'],
                         mode='markers',
                         name='Resistance Levels',
                         marker=dict(color='black', size=5)))

fig.update_layout(title='Stock Price Analysis with Support and Resistance Levels',
                  xaxis_rangeslider_visible=False)

fig.show()

In [ ]:
#Marking Support and Resistance Levels and Chart (Line)
support_levels = data[(data['Low'].shift(1) > data['Low']) & (data['Low'].shift(-1) > data['Low'])]

resistance_levels = data[(data['High'].shift(1) < data['High']) & (data['High'].shift(-1) < data['High'])]

fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'])])

fig.add_trace(go.Scatter(x=support_levels['Date'],
                         y=support_levels['Low'],
                         mode='lines',
                         name='Support Levels',
                         line=dict(color='green')))

fig.add_trace(go.Scatter(x=resistance_levels['Date'],
                         y=resistance_levels['High'],
                         mode='lines',
                         name='Resistance Levels',
                         line=dict(color='red')))

fig.update_layout(title='Stock Price Analysis with Support and Resistance Levels',
                  xaxis_rangeslider_visible=False)

fig.show()

In [ ]:
#Calculating The RSI
lookback_period = 14

price_changes = data['Close'].diff()

upward_movements = price_changes.clip(lower=0)
downward_movements = -price_changes.clip(upper=0)

average_gain = upward_movements.ewm(com=lookback_period - 1, min_periods=lookback_period).mean()
average_loss = downward_movements.ewm(com=lookback_period - 1, min_periods=lookback_period).mean()

relative_strength = average_gain / average_loss

rsi = 100 - (100 / (1 + relative_strength))

data['RSI'] = rsi

print(data[['Date', 'Close', 'RSI']])

          Date       Close        RSI
0   2022-06-06  463.700012        NaN
1   2022-06-07  463.399994        NaN
2   2022-06-08  471.299988        NaN
3   2022-06-09  466.950012        NaN
4   2022-06-10  461.850006        NaN
..         ...         ...        ...
243 2023-05-30  592.799988  65.982995
244 2023-05-31  579.849976  54.103341
245 2023-06-01  582.750000  56.013238
246 2023-06-02  587.200012  58.843408
247 2023-06-05  587.200012  58.843408

[248 rows x 3 columns]


<ipython-input-10-f940c828160a>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [ ]:
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'],
                                     name='Candlestick')])

fig.add_trace(go.Scatter(x=data['Date'],
                         y=data['RSI'],
                         mode='lines',
                         name='RSI'))


fig.update_layout(title='Stock Price Analysis with RSI',
                  xaxis_rangeslider_visible=False)

fig.show()


In [ ]:
# Define the parameters for the Bollinger Bands
window = 20  # Number of periods for moving average
std_dev = 2  # Number of standard deviations for the bands

# Calculate the rolling mean and standard deviation
rolling_mean = data['Close'].rolling(window=window).mean()
rolling_std = data['Close'].rolling(window=window).std()

# Calculate the upper and lower Bollinger Bands
upper_band = rolling_mean + (rolling_std * std_dev)
lower_band = rolling_mean - (rolling_std * std_dev)

# Create a candlestick chart
fig = go.Figure(data=[go.Candlestick(x=data['Date'],
                                     open=data['Open'],
                                     high=data['High'],
                                     low=data['Low'],
                                     close=data['Close'],
                                     name='Candlestick')])

# Add the upper and lower Bollinger Bands
fig.add_trace(go.Scatter(x=data['Date'], y=upper_band, mode='lines', name='Upper Bollinger Band',line=dict(color='blue')))
fig.add_trace(go.Scatter(x=data['Date'], y=lower_band, mode='lines', name='Lower Bollinger Band',line=dict(color='blue')))

# Customize the chart layout
fig.update_layout(title='Stock Price Analysis with Bollinger Bands',
                  xaxis_rangeslider_visible=False)

# Display the chart
fig.show()